In [1]:

import os
import uuid
import base64
from IPython import display
from unstructured.partition.pdf import partition_pdf
# from langchain.chat_models import ChatOpenAI
from langchain_community.chat_models import ChatOllama
from langchain_community import embeddings

# from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.schema.messages import HumanMessage, SystemMessage
from langchain.schema.document import Document
from langchain.vectorstores import FAISS
from langchain.retrievers.multi_vector import MultiVectorRetriever

In [5]:
output_path = "./test2sri"


In [6]:
raw_pdf_elements = partition_pdf(
    filename=r"C:\Users\Public\VS_Code_Workspace\Langchain_LLM\sriram\chatbot\chatbot\IF10244.pdf",
    extract_images_in_pdf=True,
    infer_table_structure=True,
    chunking_strategy="by_title",
    max_characters=4000,
    new_after_n_chars=3800,
    combine_text_under_n_chars=2000,
    extract_image_block_output_dir=output_path,
)

This function will be deprecated in a future release and `unstructured` will simply use the DEFAULT_MODEL from `unstructured_inference.model.base` to set default model name
Some weights of the model checkpoint at microsoft/table-transformer-structure-recognition were not used when initializing TableTransformerForObjectDetection: ['model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSe

In [8]:
text_elements = []
table_elements = []

text_summaries = []
table_summaries = []

summary_prompt = """
Summarize the following {element_type}:
{element}
"""
summary_chain = LLMChain(
    llm=ChatOllama(model="mistral",temperature=0 ),
    prompt=PromptTemplate.from_template(summary_prompt)
)

for e in raw_pdf_elements:
    if 'CompositeElement' in repr(e):
        text_elements.append(e.text)
        summary = summary_chain.run({'element_type': 'text', 'element': e})
        text_summaries.append(summary)

    elif 'Table' in repr(e):
        table_elements.append(e.text)
        summary = summary_chain.run({'element_type': 'table', 'element': e})
        table_summaries.append(summary)

In [5]:
table_elements[0]

'2018 2019 2020 Number of Fires (thousands) Federal 12.5 10.9 14.4 FS 5.6 5.3 6.7 DOI 7.0 5.3 7.6 2021 14.0 6.2 7.6 2022 11.7 5.9 5.8 Other 0.1 0.2 <0.1 0.2 0.1 Nonfederal 45.6 39.6 44.6 45.0 57.2 Total 58.1 Acres Burned (millions) Federal 4.6 FS 2.3 DOI 2.3 50.5 3.1 0.6 2.3 59.0 7.1 4.8 2.3 59.0 5.2 4.1 1.0 69.0 4.0 1.9 2.1 Other <0.1 <0.1 <0.1 <0.1 Nonfederal 4.1 1.6 3.1 1.9 Total 8.8 4.7 10.1 7.1 <0.1 3.6 7.6'

In [9]:
table_summaries[0]

" The table presents data on the number of fires and acres burned for various categories in each of the years 2018 to 2022. Here's a summary:\n\nNumber of Fires:\n- Federal fires ranged from 10.9 thousand in 2019 to 14.4 thousand in 2020.\n- FS (Forest Service) fires ranged from 5.3 thousand in 2019 to 6.7 thousand in 2020.\n- DOI (Department of the Interior) fires ranged from 5.3 thousand in 2019 to 7.6 thousand in 2020.\n- Other and nonfederal fires had very low numbers of incidents in all years.\n\nAcres Burned:\n- Federal acres burned ranged from 4.1 million in 2018 to 59.0 million in 2021.\n- FS acres burned ranged from 2.3 million in 2018 to 7.6 million in 2021.\n- DOI acres burned ranged from 0.6 million in 2018 to 4.1 million in 2021.\n- Other and nonfederal categories had negligible acres burned in all years.\n\nTotal fires and acres burned for all categories combined were highest in 2021, with 14.0 thousand fires and 69.0 million acres burned. The lowest total occurred in 201

In [10]:
table_summaries[1]

' The table presents data on the number of structures burned in different years and the percentage of those structures that were residences. In 2019, a total of 963 structures were burned, out of which 46% were residences. In 2020, the number of structures burned increased significantly to 17,904, with 54% being residences. The number of structures burned decreased in 2021 to 5,972, and the percentage of residences among them was 46%. In 2022, only 2,717 structures were burned, with 46% of them being residences. Overall, there is a clear trend of an increase in the number of structures burned from 2019 to 2020, followed by a significant decrease in 2021 and 2022. The percentage of residences among the structures burned remained relatively consistent across the years.'

In [11]:
image_elements = []

def encode_image(image_path):
    with open(image_path, "rb") as f:
        return base64.b64encode(f.read()).decode('utf-8')
for i in os.listdir(output_path):
    if i.endswith(('.png', '.jpg', '.jpeg')):
        image_path = os.path.join(output_path, i)
        encoded_image = encode_image(image_path)
        image_elements.append(encoded_image)


In [12]:
image_elements[0]

'...'

In [13]:
image_elements[2]

'...'

In [14]:
import os
import ollama
from tqdm import tqdm
import base64
from PIL import Image
from io import BytesIO

# Initialize an empty list to store the results
img_summaries = []


# Iterate over all image strings in the list with a tqdm progress bar
for image_string in tqdm(image_elements, desc="Processing images"):
    # Decode the image string using utf-8 encoders
    img_data = base64.b64decode(image_string)
    img = Image.open(BytesIO(img_data))
    
    # Save the image to a temporary file
    img.save("temp.jpg")
    
    # Call the chat function with the image file
    res = ollama.chat(
        model="llava",
        messages=[
            {
                'role':'user',
                'content':'Describe this image elaborativly:',
                'images':["temp.jpg"]
            }
        ]
    )
    
    # Append the result to the image_summary list
    img_summaries.append(res['message']['content'])

# Print the image_summary list
print(img_summaries)


Processing images: 100%|██████████| 5/5 [01:44<00:00, 20.99s/it]

[' The image is a photograph of what appears to be a computer screen displaying a website or a digital document. The main focus of the image is on the text displayed, which reads:\n\n"CONGRESSIONAL PROCEDURE SERVICE\nInforming lawmakers since 1805"\n\nBelow this, there\'s a subheading that states: "Funding for this website comes from the people of the United States."\n\nThe style of the image is informational and professional, typical of an official government service. The text is in bold, capitalized letters, emphasizing its importance. The background is a neutral grey color, which is common for web pages to provide contrast and readability for the text. ', ' The image is a composite of two different graphs, each providing data on wildfires in the United States.\n\nThe upper graph is a line chart titled "Fires thousands, acres burned, millions." It displays three lines that represent these categories over time. The x-axis of this chart is labeled with years from 1990 to 2023, and the 

In [15]:
img_summaries[0]

' The image is a photograph of what appears to be a computer screen displaying a website or a digital document. The main focus of the image is on the text displayed, which reads:\n\n"CONGRESSIONAL PROCEDURE SERVICE\nInforming lawmakers since 1805"\n\nBelow this, there\'s a subheading that states: "Funding for this website comes from the people of the United States."\n\nThe style of the image is informational and professional, typical of an official government service. The text is in bold, capitalized letters, emphasizing its importance. The background is a neutral grey color, which is common for web pages to provide contrast and readability for the text. '

In [21]:
img_summaries[1]

' The image is a graphical representation of data, likely related to a specific event or trend. It appears to be a screenshot taken from a digital device displaying the data in a visual format that includes both text and numbers.\n\nAt the top of the image, there\'s a title "Fires" with two subtitles: "thousands" on the left side and "burned, millions" on the right. This suggests that the graph is tracking some measure of fires in terms of thousands and millions burned.\n\nBelow the title, there are two pieces of data represented on a horizontal axis that seems to represent time, likely years based on the context provided by the subtitles. On the left side of the image, there\'s a vertical bar chart representing the number of fires in thousands per year. The bars show varying numbers of fires each year, with some years showing higher numbers and others lower.\n\nOn the right side, there is another line graph that represents the total area burned by fires over time. This graph has two l

In [22]:
img_summaries[2]

' The image displays a bar chart, which is overlaid on a table. The chart presents data for the number of fires and the amount of land burned (in square kilometers) from 2016 to 2020. On the left side of the chart, there are two bars for each year: one bar indicates the number of fires, and the other shows the area burned in square kilometers.\n\nTo the right of the chart, there is a table with columns representing years from 2016 to 2020, and rows representing data sources or types. The row headers are "Number of fires," "Area burned (in square km)," and "Fires per million."\n\nBelow the chart and table, there is a caption that reads "Year." Across the top of the image, there is a horizontal bar with text indicating different time periods: "2016 to 2020," "2015 to 2020," "2014 to 2020," and "2013 to 2020."\n\nIn the top right corner of the image, there is a key with symbols indicating different data categories: fires per million, area burned (in square km), and fires. The legend sugge

In [16]:
documents = []
retrieve_contents = []

for e, s in zip(text_elements, text_summaries):
    i = str(uuid.uuid4())
    doc = Document(
        page_content = s,
        metadata = {
            'id': i,
            'type': 'text',
            'original_content': e
        }
    )
    retrieve_contents.append((i, e))
    documents.append(doc)

for e, s in zip(table_elements, table_summaries):
    doc = Document(
        page_content = s,
        metadata = {
            'id': i,
            'type': 'table',
            'original_content': e
        }
    )
    retrieve_contents.append((i, e))
    documents.append(doc)

for e, s in zip(image_elements, img_summaries):
    doc = Document(
        page_content = s,
        metadata = {
            'id': i,
            'type': 'image',
            'original_content': e
        }
    )
    retrieve_contents.append((i, s))
    documents.append(doc)

vectorstore = FAISS.from_documents(documents=documents, embedding=embeddings.OllamaEmbeddings(model='nomic-embed-text'))

In [26]:
vectorstore.save_local("sri_faiss")

In [2]:
embeddings = embeddings.OllamaEmbeddings(model='nomic-embed-text')


In [3]:
db = FAISS.load_local("sri_faiss", embeddings,allow_dangerous_deserialization=True)

In [4]:
prompt_template = """
You are a data scientist and an expert in analyzing images and tables and text.
Answer the question based only on the following context, which can include text, images and tables:
{context}
Question: {question}
Don't answer if you are not sure and decline to answer and say "Sorry, I don't have much information about it."
Just return the helpful answer in as much as detailed possible.
Answer:
"""

In [5]:
qa_chain = LLMChain(llm=ChatOllama(model="mistral", temperature=0),
                        prompt=PromptTemplate.from_template(prompt_template))

In [10]:
db.similarity_search("Annual Wildfires image")

[Document(page_content=' The image is a graphical representation of data, likely related to a specific event or trend. It appears to be a screenshot taken from a digital device displaying the data in a visual format that includes both text and numbers.\n\nAt the top of the image, there\'s a title "Fires" with two subtitles: "thousands" on the left side and "burned, millions" on the right. This suggests that the graph is tracking some measure of fires in terms of thousands and millions burned.\n\nBelow the title, there are two pieces of data represented on a horizontal axis that seems to represent time, likely years based on the context provided by the subtitles. On the left side of the image, there\'s a vertical bar chart representing the number of fires in thousands per year. The bars show varying numbers of fires each year, with some years showing higher numbers and others lower.\n\nOn the right side, there is another line graph that represents the total area burned by fires over tim

In [6]:

def answer(question):
    relevant_docs = db.similarity_search(question)
    context = ""
    relevant_images = []
    for d in relevant_docs:
        if d.metadata['type'] == 'text':
            context += '[text]' + d.metadata['original_content']
        elif d.metadata['type'] == 'table':
            context += '[table]' + d.metadata['original_content']
        elif d.metadata['type'] == 'image':
            context += '[image]' + d.page_content
            relevant_images.append(d.metadata['original_content'])
    result = qa_chain.run({'context': context, 'question': question})
    return result, relevant_images

In [7]:
result, relevant_images = answer("tell me the insight about Annual Wildfires and Acres Burned, 1993-2022?")
print(result)

c:\Users\Public\VS_Code_Workspace\Langchain_LLM\llm_ground\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


 According to data compiled by the National Interagency Coordination Center (NICC), the number of annual wildfires in the United States has decreased slightly over the last 30 years, but the number of acres affected annually has generally increased. From 2000 to 2022, an average of 70,025 wildfires burned an average of 7.0 million acres per year. The majority of these fires were human-caused (89% on average), and most of the acreage burned was in the Western states. In 2022, there were 68,988 wildfires that burned 7.6 million acres, with over 20,000 of those fires occurring in the West and impacting approximately 5.8 million acres. The highest levels of resource commitment to wildfire suppression occurred in 2021, when the nationwide Preparedness Level reached the maximum level for 68 days. In 2022, there were over 2,700 structures burned in wildfires, with the majority of the damage occurring in California.


In [8]:
len(relevant_images)

0

In [14]:
display.display(display.Image(base64.b64decode(relevant_images[0])))

IndexError: list index out of range